## Initialization

In [ ]:
#@title
! git clone https://github.com/proroklab/VectorizedMultiAgentSimulator.git

In [ ]:
#@title
%cd /content/VectorizedMultiAgentSimulator

!pip install -r requirements.txt
!apt-get update
!apt-get install -y x11-utils 
!apt-get install -y xvfb
!apt-get install -y imagemagick
!pip install -e .

In [ ]:
#@title
!pip install pyvirtualdisplay
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

## Run


In [ ]:
import torch
import os
has_gpu = torch.cuda.is_available()
os.environ["RLLIB_NUM_GPUS"] = "1" if has_gpu else "0"

In [6]:
#  Copyright (c) 2022.
#  ProrokLab (https://www.proroklab.org/)
#  All rights reserved.

import os
from typing import Dict, Optional

import numpy as np
import ray
from ray import tune
from ray.rllib import BaseEnv, Policy, RolloutWorker
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.evaluation import Episode, MultiAgentEpisode
from ray.rllib.utils.typing import PolicyID
from ray.tune import register_env

import wandb
from vmas import make_env, Wrapper

scenario_name = "balance"

# Scenario specific variables.
# When modifying this also modify env_config and env_creator
n_agents = 4

# Common variables
continuous_actions = True
max_steps = 200
num_vectorized_envs = 96
num_workers = 1
vmas_device = "cpu"  # or cuda


def env_creator(config: Dict):
    env = make_env(
        scenario_name=config["scenario_name"],
        num_envs=config["num_envs"],
        device=config["device"],
        continuous_actions=config["continuous_actions"],
        wrapper=Wrapper.RLLIB,
        max_steps=config["max_steps"],
        # Scenario specific variables
        n_agents=config["n_agents"],
    )
    return env


if not ray.is_initialized():
    ray.init()
    print("Ray init!")
register_env(scenario_name, lambda config: env_creator(config))


class EvaluationCallbacks(DefaultCallbacks):
    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        episode: MultiAgentEpisode,
        **kwargs,
    ):
        info = episode.last_info_for()
        for a_key in info.keys():
            for b_key in info[a_key]:
                try:
                    episode.user_data[f"{a_key}/{b_key}"].append(info[a_key][b_key])
                except KeyError:
                    episode.user_data[f"{a_key}/{b_key}"] = [info[a_key][b_key]]

    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[str, Policy],
        episode: MultiAgentEpisode,
        **kwargs,
    ):
        info = episode.last_info_for()
        for a_key in info.keys():
            for b_key in info[a_key]:
                metric = np.array(episode.user_data[f"{a_key}/{b_key}"])
                episode.custom_metrics[f"{a_key}/{b_key}"] = np.sum(metric).item()


class RenderingCallbacks(DefaultCallbacks):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.frames = []

    def on_episode_step(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Optional[Dict[PolicyID, Policy]] = None,
        episode: Episode,
        **kwargs,
    ) -> None:
        self.frames.append(base_env.vector_env.try_render_at(mode="rgb_array"))

    def on_episode_end(
        self,
        *,
        worker: RolloutWorker,
        base_env: BaseEnv,
        policies: Dict[PolicyID, Policy],
        episode: Episode,
        **kwargs,
    ) -> None:
        vid = np.transpose(self.frames, (0, 3, 1, 2))
        episode.media["rendering"] = wandb.Video(vid, fps=1 / base_env.vector_env.env.world.dt, format="mp4")
        self.frames = []


def train():

    RLLIB_NUM_GPUS = int(os.environ.get("RLLIB_NUM_GPUS", "0"))
    num_gpus = 0.001 if RLLIB_NUM_GPUS > 0 else 0  # Driver GPU
    num_gpus_per_worker = (
        (RLLIB_NUM_GPUS - num_gpus) / (num_workers + 1) if vmas_device == "cuda" else 0
    )

    tune.run(
        PPOTrainer,
        stop={"training_iteration": 400},
        checkpoint_freq=1,
        keep_checkpoints_num=2,
        checkpoint_at_end=True,
        checkpoint_score_attr="episode_reward_mean",
        # callbacks=[
        #     WandbLoggerCallback(
        #        project=f"{scenario_name}",
        #        api_key="",
        #    )
        # ],
        config={
            "seed": 0,
            "framework": "torch",
            "env": scenario_name,
            "kl_coeff": 0.01,
            "kl_target": 0.01,
            "lambda": 0.9,
            "clip_param": 0.2,
            "vf_loss_coeff": 1,
            "vf_clip_param": float("inf"),
            "entropy_coeff": 0,
            "train_batch_size": 60000,
            "rollout_fragment_length": 125,
            "sgd_minibatch_size": 4096,
            "num_sgd_iter": 40,
            "num_gpus": num_gpus,
            "num_workers": num_workers,
            "num_gpus_per_worker": num_gpus_per_worker,
            "num_envs_per_worker": num_vectorized_envs,
            "lr": 5e-6,
            "gamma": 0.99,
            "use_gae": True,
            "use_critic": True,
            "batch_mode": "truncate_episodes",
            "env_config": {
                "device": vmas_device,
                "num_envs": num_vectorized_envs,
                "scenario_name": scenario_name,
                "continuous_actions": continuous_actions,
                "max_steps": max_steps,
                # Scenario specific variables
                "n_agents": n_agents,
            },
            "evaluation_interval": 5,
            "evaluation_duration": 1,
            "evaluation_num_workers": 0,
            "evaluation_parallel_to_training": False,
            "evaluation_config": {
                "num_envs_per_worker": 1,
                "env_config": {
                    "num_envs": 1,
                },
                # "callbacks": MultiCallbacks([RenderingCallbacks, EvaluationCallbacks]),
            },
            "callbacks": EvaluationCallbacks,
        },
    )


if __name__ == "__main__":
    train()

(pid=44668) /opt/anaconda3/envs/env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
(pid=44668)   'nearest': pil_image.NEAREST,
(pid=44668) /opt/anaconda3/envs/env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
(pid=44668)   'bilinear': pil_image.BILINEAR,
(pid=44668) /opt/anaconda3/envs/env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
(pid=44668)   'bicubic': pil_image.BICUBIC,
(pid=44668) /opt/anaconda3/envs/env/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is deprecated and will be removed in Pillow 10

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_balance_591fb_00000,14160000,"{'num_env_steps_sampled': 14160000, 'num_env_steps_trained': 14160000, 'num_agent_steps_sampled': 14160000, 'num_agent_steps_trained': 14160000}","{'rewards/0_mean': 99.64384879943259, 'rewards/0_min': -6.2609405517578125, 'rewards/0_max': 141.73487854003906, 'rewards/1_mean': 99.64384879943259, 'rewards/1_min': -6.2609405517578125, 'rewards/1_max': 141.73487854003906, 'rewards/2_mean': 99.64384879943259, 'rewards/2_min': -6.2609405517578125, 'rewards/2_max': 141.73487854003906, 'rewards/3_mean': 99.64384879943259, 'rewards/3_min': -6.2609405517578125, 'rewards/3_max': 141.73487854003906, 'agent 0/pos_rew_mean': 99.64384879943259, 'agent 0/pos_rew_min': -6.2609405517578125, 'agent 0/pos_rew_max': 141.73487854003906, 'agent 0/ground_rew_mean': 0.0, 'agent 0/ground_rew_min': 0.0, 'agent 0/ground_rew_max': 0.0, 'agent 1/pos_rew_mean': 99.64384879943259, 'agent 1/pos_rew_min': -6.2609405517578125, 'agent 1/pos_rew_max': 141.73487854003906, 'agent 1/ground_rew_mean': 0.0, 'agent 1/ground_rew_min': 0.0, 'agent 1/ground_rew_max': 0.0, 'agent 2/pos_rew_mean': 99.64384879943259, 'agent 2/pos_rew_min': -6.2609405517578125, 'agent 2/pos_rew_max': 141.73487854003906, 'agent 2/ground_rew_mean': 0.0, 'agent 2/ground_rew_min': 0.0, 'agent 2/ground_rew_max': 0.0, 'agent 3/pos_rew_mean': 99.64384879943259, 'agent 3/pos_rew_min': -6.2609405517578125, 'agent 3/pos_rew_max': 141.73487854003906, 'agent 3/ground_rew_mean': 0.0, 'agent 3/ground_rew_min': 0.0, 'agent 3/ground_rew_max': 0.0}",2023-01-10_17-20-03,False,197.441,{},141.735,99.6438,-6.26094,311,71095,92d3e75facb64bf3a6a5db4ec0551b6e,wosersysydeMacBook-Pro.local,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 20.392005970648356, 'cur_kl_coeff': 6.681911775230489e-54, 'cur_lr': 5e-06, 'total_loss': 2.9767663796033177, 'policy_loss': 0.0017569990937772672, 'vf_loss': 2.9750093873058048, 'vf_explained_var': 0.9874570196228368, 'kl': 0.0038214488556637402, 'entropy': 1.807006427007062, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 4096.0, 'num_grad_updates_lifetime': 131880.5, 'diff_num_grad_updates_vs_sampler_policy': 279.5}}, 'num_env_steps_sampled': 14160000, 'num_env_steps_trained': 14160000, 'num_agent_steps_sampled': 14160000, 'num_agent_steps_trained': 14160000}",236,127.0.0.1,14160000,14160000,14160000,60000,14160000,60000,0,1,0,0,60000,"{'cpu_util_percent': 24.53402777777778, 'ram_util_percent': 64.95416666666668}",44650,{},{},{},"{'mean_raw_obs_processing_ms': 44.55475229622829, 'mean_inference_ms': 3.412697662916898, 'mean_action_processing_ms': 19.299814500179682, 'mean_env_wait_ms': 29.72847054005817, 'mean_env_render_ms': 0.0}",28554.7,101.724,28554.7,"{'training_iteration_time_ms': 99196.919, 'load_time_ms': 27.45, 'load_throughput': 2185772.317, 'learn_time_ms': 49205.141, 'learn_throughput': 1219.385, 'synch_weights_time_ms': 2.088}",1673342403,0,14160000,236,591fb_00000,9.7637


2023-01-10 17:20:26,549	WARNING tune.py:691 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-01-10 17:20:28,786	ERROR tune.py:758 -- Trials did not complete: [PPO_balance_591fb_00000]
2023-01-10 17:20:28,787	INFO tune.py:763 -- Total run time: 28636.93 seconds (28636.69 seconds for the tuning loop).
2023-01-10 17:20:28,788	WARNING tune.py:769 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
